### Проектный практикум: Классификация с использованием набора данных об академической успеваемости

ydata_profiling - быстрый анализ DA. Это первый этап разработки
from ydata_profiling import ProfileReport

Коэфицинеты: Пирсона, Спирмена, Кендала, ...

df.phik_matrix()


data leakage - учтечка данных (при расчете на полной выборке до разделения на трейн и тест)
Тест нельзя включать никогда !!! Иначе модель подсмотрит в правильные данные

коэф детерминации используется для оценки качества модели = сумм ошибок^2 / среднеее


**Установка пакетов**

In [30]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, ElasticNet
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.pipeline import make_pipeline
!pip install phik
import phik
from phik.report import plot_correlation_matrix

**1. Загрузка данных**

In [8]:
df_train = pd.read_csv('./data/train.csv')
# df_test = pd.read_csv('./data/test.csv')

# print(df_train.info())
# print(df_train.describe())
# print(df_train)
df_train

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


**Удаление лишних столбцов**

Уникальный идентификатор:
- id

Data leakage
- Curricular units 1st sem (credited) 
- Curricular units 1st sem (enrolled) 
    'Curricular units 1st sem (evaluations)', 
    'Curricular units 1st sem (approved)', 
    'Curricular units 1st sem (grade)', 
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (credited)', 
    'Curricular units 2nd sem (enrolled)', 
    'Curricular units 2nd sem (evaluations)', 
    'Curricular units 2nd sem (approved)', 
    'Curricular units 2nd sem (grade)', 
    'Curricular units 2nd sem (without evaluations)',

Данные не относящиеся к обучению
- 'Unemployment rate',
    'Inflation rate',
    'GDP',

Колонка	Причина
Application order	Порядок подачи заявки обычно не влияет на результат обучения/успех.
Nacionality	Если числовой код страны не несёт полезной информации и нет планов на dummy encoding, может быть удалён.
Displaced	Скорее административная метка, маловероятно влияет на оценку.
Educational special needs	Очень мало случаев (mean ~0.0037), почти все 0 → мало информативно.
Debtor	Малое количество 1, может вносить шум, если не связано с целевой переменной.
Tuition fees up to date	Почти все 1 → низкая вариативность, мало информации для модели.
International	Почти все 0 → низкая вариативность.

In [9]:
cols_to_drop = [
    'id', 
    'Curricular units 1st sem (credited)', 
    'Curricular units 1st sem (enrolled)', 
    'Curricular units 1st sem (evaluations)', 
    'Curricular units 1st sem (approved)', 
    'Curricular units 1st sem (grade)', 
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (credited)', 
    'Curricular units 2nd sem (enrolled)', 
    'Curricular units 2nd sem (evaluations)', 
    'Curricular units 2nd sem (approved)', 
    'Curricular units 2nd sem (grade)', 
    'Curricular units 2nd sem (without evaluations)',
    'Unemployment rate',
    'Inflation rate',
    'GDP',
    'Application order',
    'Nacionality',
    'Displaced',
    'Educational special needs',
    'Debtor',
    'Tuition',
    'International',
]

df_train = df_train.drop(columns=cols_to_drop, errors='ignore')
# print(df_train.info)
# df_train

print(df_train.shape)
print(df_train.describe())

(76518, 16)
       Marital status  Application mode        Course  \
count    76518.000000      76518.000000  76518.000000   
mean         1.111934         16.054419   9001.286377   
std          0.441669         16.682337   1803.438531   
min          1.000000          1.000000     33.000000   
25%          1.000000          1.000000   9119.000000   
50%          1.000000         17.000000   9254.000000   
75%          1.000000         39.000000   9670.000000   
max          6.000000         53.000000   9991.000000   

       Daytime/evening attendance  Previous qualification  \
count                76518.000000            76518.000000   
mean                     0.915314                3.658760   
std                      0.278416                8.623774   
min                      0.000000                1.000000   
25%                      1.000000                1.000000   
50%                      1.000000                1.000000   
75%                      1.000000              

**2. Составление быстрого отчета**

In [ ]:
# !pip install ydata_profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(df_train, title="Отчет по анализу")
profile.to_file("profile_report.html")
# profile

Удаление столбцов с большой корреляцией (Мультиколлинеарность)

In [11]:
# Previous qualification (grade)
# International

coll_by_multicollinearity = [
    'Previous qualification (grade)',
    'International'
]
df_train = df_train.drop(coll_by_multicollinearity, errors='ignore')
print(df_train.shape)
df_train

(76518, 16)


,Marital status,Application mode,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,Target
0,1,1,9238,1,1,126.0,1,19,5,5,122.6,1,0,1,18,Graduate
1,1,17,9238,1,1,125.0,19,19,9,9,119.8,1,0,0,18,Dropout
2,1,17,9254,1,1,137.0,3,19,2,3,144.7,1,1,0,18,Dropout
3,1,1,9500,1,1,131.0,19,3,3,2,126.1,1,0,1,18,Enrolled
4,1,1,9500,1,1,132.0,19,37,4,9,120.1,1,0,0,18,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,17,9254,1,1,121.0,19,1,7,5,116.5,1,0,1,18,Graduate
76514,1,1,9254,1,1,125.0,1,38,4,9,131.6,1,0,0,19,Graduate
76515,5,17,9085,1,1,138.0,37,37,9,10,123.3,1,0,0,19,Enrolled
76516,1,1,9070,1,1,136.0,38,37,5,9,124.8,1,0,0,18,Dropout


Используем phink для обнаружения нелинейных корреляций

In [28]:
# search_list = ['Marital status', 'Application mode', 'Course', 'Daytime/evening attendance', 'Previous qualification', 'Previous qualification (grade)', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation", 'Admission grade', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment'] 
corr_matrix = df_train.phik_matrix()
plot_correlation_matrix(corr_matrix.values, x_labels=corr_matrix.columns, y_labels=corr_matrix.index, figsize=(10, 8))

interval columns not set, guessing: ['Marital status', 'Application mode', 'Course', 'Daytime/evening attendance', 'Previous qualification', 'Previous qualification (grade)', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation", 'Admission grade', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment', 'Target_reg']


**2. Анализ данных**
- выбор признаков
- масштабирование

In [29]:
# Создаём маппинг
target_mapping = {
    'Dropout': 0,
    'Enrolled': 1,
    'Graduate': 2
}

# Применяем к данным
df_train['Target_reg'] = df_train['Target'].map(target_mapping)

# Разделение данных
X = df_train.drop(['Target', 'Target_reg'], axis=1)  # удаляем оба столбца
y = df_train['Target_reg']  # ← числовая цель!

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # ← критически важно для классификации!
)

# Создаем пайплайн
pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Обучаем пайплайн
pipeline.fit(X_train, y_train)

# Создаем KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Выполняем кросс-валидацию
cv_metrics = cross_validate(
    estimator=pipeline,
    X=X_train,
    y=y_train,
    cv=kf,
    scoring='r2',
# scoring='accuracy',
    return_train_score=True,
    n_jobs=-1
)

print("Средняя точность на кросс-валидации:", cv_metrics['test_score'].mean())
# cv_metrics


Средняя точность на кросс-валидации: 0.3821683640524142


Посмотреть на результат всех преобразованйи можно так:

Берем срез `[:-1]`, чтобы исключить последний шаг пайплайна, который  является моделью (LinearRegression).

In [18]:
X_train_transformed = pipeline[:-1].transform(X_train)
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=X_train.columns)
X_train_transformed_df

,Marital status,Application mode,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment
0,-0.254584,0.061213,0.471810,0.304585,-0.306715,-2.224769,1.113226,0.910038,-0.088512,-0.110467,-1.442024,0.342970,-0.678256,-0.573594,-0.329583
1,-0.254584,1.381216,0.370559,0.304585,-0.306715,1.604316,-0.899956,-0.633137,-0.490876,-0.527696,-1.465933,0.342970,1.474369,-0.573594,0.540879
2,-0.254584,0.061213,0.081193,0.304585,-0.306715,1.604316,-1.224663,-1.371177,0.026449,-0.408488,0.526476,0.342970,-0.678256,-0.573594,-0.184506
3,-0.254584,-0.898789,0.140394,0.304585,-0.306715,-0.583733,-0.055719,-0.297664,-0.260954,-0.110467,-0.676939,0.342970,-0.678256,-0.573594,-0.474660
4,-0.254584,-0.898789,0.370559,0.304585,-0.306715,-0.036720,-0.510308,0.910038,-0.203473,-0.348883,-0.445819,-2.915703,-0.678256,-0.573594,-0.474660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57383,-0.254584,0.061213,0.276501,0.304585,-0.306715,-0.583733,1.178167,0.977132,-0.203473,-0.050863,-0.477698,0.342970,-0.678256,1.743392,-0.329583
57384,-0.254584,-0.898789,0.081193,0.304585,-0.306715,-0.219058,-1.224663,-0.297664,-0.318434,-0.110467,-0.501607,0.342970,-0.678256,-0.573594,-0.474660
57385,-0.254584,-0.058787,0.071787,0.304585,-0.306715,1.604316,-1.159722,-1.371177,-0.031032,-0.050863,1.961011,0.342970,-0.678256,-0.573594,-0.619737
57386,-0.254584,1.381216,-0.545675,-3.283161,-0.306715,0.063565,1.113226,0.910038,0.026449,-0.229675,-1.944111,0.342970,-0.678256,-0.573594,1.846572


И оцениваем качество:

In [19]:
print(f"Среднее качество на тренировочной выборке: {cv_metrics['train_score'].mean()}")
print(f"Среднее качество на валидации: {cv_metrics['test_score'].mean()}")

Среднее качество на тренировочной выборке: 0.6518784586648468
Среднее качество на валидации: 0.6507981507347548
